#Search
This is the function that will return items that meet the requirements the user provided.

## Multiple Vector Search with ranking.

Ok, how to surface the best matches? We have simplified the process of outlining requirements, by separating out items we want to look for. However, now we need to combine those outputs into one single listing. 

## Reciprocal Ranked Fusion (RRF)

We perform a search across every keyword item. Then, we give each result a score based on the number of times it appears in the results. 

Some keywords will be marked 'absolute' - meaning that not being present in that keyword's list drops the item from the results entirely. 

In [1]:
#| export
from hn_jobs_chat.keys import keys

def format_job_result(job_listing):
    job = {}
    
    for key in keys:
        if key in job_listing:
            job[key] = job_listing[key]

    return job

In [2]:
#| export 
def format_results(results):
    formatted_results = []

    for job_listing in results:
        job = format_job_result(job_listing[0])

        formatted_results.append(job)

    return formatted_results


In [3]:

results = [[{ 'job_title': 'Software Engineer', 'company': 'Google', 'location': 'Mountain View, CA' }]]

format_results(results)

[{'company': 'Google',
  'location': 'Mountain View, CA',
  'job_title': 'Software Engineer'}]

In [4]:
from hn_jobs_chat.keys import described_keys

print(described_keys[0]['type'])


string


In [6]:
#| default_exp search
#| export
import psycopg2
from hn_jobs_chat.keys import described_keys_dict
from hn_jobs_chat.globals import postsTableName

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

def search(requirements):
    initial_results = {}

    for key in requirements.keys():
        
        if described_keys_dict[key]['type'] == 'string':
            query = f"SELECT row_to_json({postsTableName}) FROM {postsTableName} WHERE {key} = '{requirements[key]}' LIMIT 40;"

            cursor.execute(query)
            results = cursor.fetchall()
            formatted_results = format_results(results)
            initial_results[key] = formatted_results

        elif described_keys_dict[key]['type'] == 'array':
            initial_results[key] = {}

            newTableName = f"{postsTableName}_{key}"

            for item in requirements[key]:
                query = f"""
                    SELECT row_to_json({postsTableName})
                    FROM {postsTableName}
                    JOIN {newTableName} ON {newTableName}.postId = {postsTableName}.id
                    WHERE {newTableName}.item = '{item}'
                    LIMIT 40;
                """

                cursor.execute(query)
                results = cursor.fetchall()
                formatted_results = format_results(results)
                initial_results[key][item] = formatted_results

        else:
            raise ValueError(f"Invalid type: {described_keys_dict[key]['type']}")

        
        # conn.commit()

    return initial_results



In [7]:
search({
    "location": "San Francisco"
})


{'location': [{'company': 'Rainbow Insurance',
   'location': 'San Francisco',
   'timezone': None,
   'industry': None,
   'target_market': None,
   'company_status': None,
   'company_description': None,
   'company_goal': 'to provide insurance policies to small businesses by evaluating them against various risks.',
   'company_stage': 'seed',
   'more_company_info': None,
   'employment_type': 'Full Time',
   'remote_or_local_details': 'Hybrid (2 days on-site in SF)',
   'job_title': 'Fullstack Software Engineer',
   'job_description': 'We are looking for an experienced software engineer to join our team and help us grow and innovate.',
   'job_requirements': '3+ years of professional experience as a software engineer,Experience in fintech is an optional but major plus,Familiarity with our tech stack (Golang, React through a Next.js app, GraphQL, PostgreSQL) is beneficial but not required',
   'job_soft_skills': None,
   'product_description': None,
   'tech_stack': None,
   'applic

In [7]:
#| hide
from nbdev import nbdev_export
nbdev_export()